In [34]:
import numpy as np
from skimage.transform import rescale, resize
from skimage.filters import gaussian
from cellpose import models
import torch
import tifffile
import nd2

import napari

In [2]:
viewer = napari.Viewer()

In [16]:
d = torch.device('mps')

# change gpu=True if on windows, and get rid of device
# model = models.Cellpose(gpu=True, model_type='cyto2')
model = models.Cellpose(gpu=False, device=d, model_type='cyto2')

### nd2 files
We will be using a Nikon file for this notebook, using the `nd2` package. The `nd2` package has an imread function like skimage and tifffile:

```python
data = nd2.imread(filename)
```

Instead, we will use the `ND2File` object gives us access to the metadata (like x, y, z scaling, excitation wavelengths, etc.) that we will need.


In [51]:
imagefile = 'Data/WT003.nd2'

### create the file object - this doesn't read the image array data
nd2file = nd2.ND2File(imagefile)
image = nd2file.asarray()
image.shape

/var/folders/p_/_q13km4n1_l_m239h13462k8s9_w4f/T/ipykernel_21345/3635283605.py:4: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  nd2file = nd2.ND2File(imagefile)


(40, 3, 2044, 2048)

Add the image to the napari viewer. From the shape output in the last cell, the channel axis is 1

In [5]:
viewer.add_image(image, channel_axis=1)

[<Image layer 'Image' at 0x3490dc130>,
 <Image layer 'Image [1]' at 0x2e53de620>,
 <Image layer 'Image [2]' at 0x348ec0d90>]

Rotate the image in 3D and notice the 3D scaling is not set. To get the right values for the z scaling we need to look at the metadata.


In [64]:
nd2file.metadata

Volume(axesCalibrated=(True, True, True), axesCalibration=(0.065, 0.065, 0.3), axesInterpretation=('distance', 'distance', 'distance'), bitsPerComponentInMemory=16, bitsPerComponentSignificant=16, cameraTransformationMatrix=(0.999993283193413, -0.003665183223043356, 0.003665183223043356, 0.999993283193413), componentCount=1, componentDataType='unsigned', voxelCount=(2048, 2044, 40), componentMaxima=[0.0], componentMinima=[0.0], pixelToStageTransformationMatrix=None)

In [10]:
md1 = nd2file.metadata.channels[1]
xum, yum, zum = md1.volume.axesCalibration
zscale = xum/zum
xum, yum, zum, zscale

(0.065, 0.065, 0.3, 0.21666666666666667)

In [11]:
for y in viewer.layers:
    y.scale = (1/zscale, 1, 1)

In [21]:
viewer.add_image(masks)

<Image layer 'masks' at 0x30ace3910>

In [28]:
scaled = rescale(image, (1, 1, .5*zscale, .5*zscale), preserve_range=True)
scaled = gaussian(scaled, sigma=(1, 0, 1, 1))
scaled.shape

(40, 3, 221, 222)

In [29]:
viewer.layers.clear()
viewer.add_image(scaled, channel_axis=1)

[<Image layer 'Image' at 0x2f0ef7130>,
 <Image layer 'Image [1]' at 0x386821e10>,
 <Image layer 'Image [2]' at 0x40d80d180>]

In [43]:
masks, _, _, _ = model.eval(scaled, diameter=40, do_3D=True, channels=[1,2],
                            cellprob_threshold=.5,
                            flow_threshold=.3) 

In [33]:
viewer.add_labels(masks)

<Labels layer 'masks' at 0x386f51c30>

In [47]:
shape = (image.shape[0], image.shape[2], image.shape[3])
smasks = resize(masks, shape, order=0, preserve_range=True)

In [49]:
viewer.layers.clear()
viewer.add_image(image, channel_axis=1)
viewer.add_labels(smasks)


<Labels layer 'smasks' at 0x3867c0070>

In [ ]:
for y in viewer.layers:
    y.scale = (1/zscale, 1, 1)